### Questions to answer
* Relation to eigenvectors?
* Why so effective at factorising? What representational power do we lose? Can we measure this?
* How does it work for tensors?
* What do U,V mean intuitively? Rotations on some sort, but to where?
* What is its computational complexity?
* 

* https://en.wikipedia.org/wiki/Higher-order_singular_value_decomposition
* https://en.wikipedia.org/wiki/Tensor_product

In [1]:
import tensorflow as tf
import numpy as np
sess = tf.InteractiveSession()

A = tf.truncated_normal((784,100))
B = tf.truncated_normal((100,45))
C = tf.truncated_normal((3,4,5))
tf.initialize_all_variables().run()

In [2]:
def eig(M):
    eig = tf.self_adjoint_eig(M)
    return eig[1:,:],eig[0,:] #eigenvalues, eigenvectors

def svd2D(x):
    xT = tf.transpose(x)
    u,l = eig(tf.matmul(x,xT)) #which eigen values?
    v,_ = eig(tf.matmul(xT,x)) #the longest set?
    return u,tf.sqrt(l),v

u,s,v  = svd2D(A)
print(u.get_shape(),s.get_shape(),v.get_shape())

(784, 784) (784,) (100, 100)


In [3]:
def isorthonormal(x):
    return tf.equal(tf.cast(x.get_shape()[0],tf.float32) , 
                    tf.trace(tf.round(tf.matmul(x,tf.transpose(x)))))

sess.run(isorthonormal(u))

True

In [4]:
def tensorprod(A,B):
    pass

$$\mathbf {M} =\sum _{i}\mathbf {A} _{i}=\sum _{i}\sigma _{i}\mathbf {U} _{i}\otimes \mathbf {V} _{i}^{\dagger }$$

* So how many times can we do this sum? We are limited by the minimum of the number of columns of U or V. Do we lose accuracy? 
* Does this mean that u and V are ordered according to the singular values?


```python
#Given some tensor
shape = [n1,n2,n3, ... ni]
A = tensor(shape)
for n in shape:
    Ai = reshape(A, [n,others]) 
    #wasteful for 2d? #transpose vs reshape?
    #how does reshape work for potnetially ambiguious shapes? [2x3x2].reshape([2,2,3])
    u,s = eig(Ai * Ai.T) #dont need right-singular vectors
    
    u
```

In [20]:
import functools

In [54]:
shape = C.get_shape()

vectors = []
for n in shape:
    others = [i for i in shape if i!= n] #not very nice...
    m = functools.reduce(lambda x,y:x*y,others) #also ugly
    
    M = tf.reshape(C,[tf.to_int32(n),tf.to_int32(m)]) #will this always work?!? what about ambiguious cases?
    u = tf.self_adjoint_eig(tf.matmul(M,tf.transpose(M)))
    vectors.append(u[1:,:])
    print(u[1:,:].get_shape())

(3, 3)
(4, 4)
(5, 5)


In [11]:
def svd(x):
    """
    n dimensional svd.
    Works recursively.
    
    Args:
    Some tensor. [n x m x ... x l]
    
    Returns:
    S,          singular values
    U,V,W, ...  eigenvectors
    """
    
    x = tf.squeeze(x) #to handle [1 x n x m]
    if len(x.get_shape()) == 2:
        return svd2D(x)[0] #just the left eigenvectors
    else:
        return svd(x[0,:,:]) , svd(x[1:,:,:])
        
sess.run(svd(C))
# why do the numbers change when I run this on the same inputs?
# the decomposition isnt unique? but that shouldnt matter
# with a deterministic algorithm?

(array([ 0.64706647,  1.47833908,  2.42983222,  3.34138894], dtype=float32),
 array([ 0.4552415 ,  1.77607119,  2.16378045,  2.71583319], dtype=float32))

In [6]:
import matplotlib.pyplot as plt
%matplotlib inline

#error hear map in 2D
#visualise transforms of U,V